In [25]:
import os
import re
import en_ner_bc5cdr_md
import spacy
import copy
import pandas as pd
import numpy as np
import evaluate
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset, DatasetDict
import torch 
from transformers import TrainingArguments, Trainer
from torch.optim import AdamW

# Set a random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
spacy.util.fix_random_seed(seed)

# SET DATA TO TEST
# "Line Data" or "Sentencizer Data"
data_to_test = "Line Data"

In [26]:
line_data = pd.read_csv(r'data\preprocessed-data\ast_line_data.csv')
line_data = line_data.rename(columns={'new_line': 'sentence'})
sentencizer_data = pd.read_csv(r'data\preprocessed-data\ast_sentencizer_data.csv')
sentencizer_data = sentencizer_data.rename(columns={'precessed': 'sentence'})
data_to_test = "Line Data"

In [27]:
def get_data(data_in):
    data = data_in.sample(frac=1).copy()
    X = data['sentence']
    y = data['label']
    X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
    X_test, X_valid, y_test, y_valid = train_test_split(X_test_valid, y_test_valid, test_size=0.5, random_state=seed, stratify=y_test_valid)
    print(f"X shape {X.shape} y shape : {y.shape}")
    print(f"X_train shape {X_train.shape} y_train shape : {y_train.shape}")
    print(f"X_val shape {X_valid.shape} y_val shape : {y_valid.shape}")
    print(f"X_test shape {X_test.shape} y_test shape : {y_test.shape}")
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test


if data_to_test == "Line Data":
    X_train, y_train, X_valid, y_valid, X_test, y_test = get_data(line_data)
elif data_to_test == "Sentencizer Data":
    X_train, y_train, X_valid, y_valid, X_test, y_test = get_data(sentencizer_data)

else:
    print(data_to_test, "not a valid choice")

X shape (18397,) y shape : (18397,)
X_train shape (14717,) y_train shape : (14717,)
X_val shape (1840,) y_val shape : (1840,)
X_test shape (1840,) y_test shape : (1840,)


In [28]:
print("Encoding Labels .....")
encoder = LabelEncoder()
encoder.fit(y_train)
y_train_encode = np.asarray(encoder.transform(y_train))
y_valid_encode = np.asarray(encoder.transform(y_valid))
y_test_encode = np.asarray(encoder.transform(y_test))

Encoding Labels .....


In [36]:
train_df = pd.DataFrame(X_train,columns=['sentence'])
valid_df = pd.DataFrame(X_valid,columns=['sentence'])
test_df = pd.DataFrame(X_test,columns=['sentence'])

train_df['label'] = y_train_encode.tolist()
valid_df['label'] = y_valid_encode.tolist()
test_df['label'] = y_test_encode.tolist()

print(train_df.head())

ds = DatasetDict ({
 'train': Dataset.from_pandas(train_df),
 'validation': Dataset.from_pandas(valid_df),
 'test': Dataset.from_pandas(test_df)
})

columns_to_remove = ['__index_level_0__']
ds['train'] = ds['train'].remove_columns(columns_to_remove)
ds['validation'] = ds['validation'].remove_columns(columns_to_remove)
ds['test'] = ds['test'].remove_columns(columns_to_remove)

                                                sentence  label
13372  s / p inferior STEMI ( 2/17 ) with stent to RC...      2
13139  On physical examination , she is an obese whit...      2
17592                   10 [entity] Psoriasis [entity] .      2
7202   Chest x-ray revealed [entity] streaky lung fie...      2
6118   4) [entity] STEROID INDUCED DIABETES MELLITUS ...      2


In [37]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 14717
    })
    validation: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1840
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1840
    })
})

In [38]:
import torch 
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

## Full and head fine tuning

In [40]:
device

device(type='cuda')

In [41]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModel 
tokenizer_clinical_bio  = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT",model_max_length=150)
model_clinical = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT", 
                                                                    num_labels=3,id2label={0: 'PRESENT', 1: 'ABSENT', 2:'POSSIBLE'})

Some weights of the model checkpoint at emilyalsentzer/Bio_Discharge_Summary_BERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from

In [42]:
model_clinical = model_clinical.to(device)

In [43]:
special_tokens_dict = {"additional_special_tokens": ["[entity]"]}
num_added_toks = tokenizer_clinical_bio.add_special_tokens(special_tokens_dict,False)

print("We have added", num_added_toks, "tokens")
# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
model_clinical.resize_token_embeddings(len(tokenizer_clinical_bio))

We have added 1 tokens


Embedding(28997, 768)

In [44]:
def tokenize_function(example):
    return tokenizer_clinical_bio(example["sentence"],   padding="max_length", truncation=True)

In [45]:
tokenized_ds = ds.map(tokenize_function, batched=True)
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds = tokenized_ds.remove_columns(["sentence"])
#tokenized_ds = tokenized_ds.remove_columns(["__index_level_0__"])
tokenized_ds.set_format("torch")

Map:   0%|          | 0/14717 [00:00<?, ? examples/s]

Map:   0%|          | 0/1840 [00:00<?, ? examples/s]

Map:   0%|          | 0/1840 [00:00<?, ? examples/s]

In [46]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14717
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1840
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1840
    })
})

In [47]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [48]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="clinbert_trainer", evaluation_strategy="epoch", learning_rate=1e-5, num_train_epochs=1,)

trainer = Trainer(
    model=model_clinical,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

C:\Users\kcaro\miniconda3\envs\clinical-adapter\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14717
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1840
  Number of trainable parameters = 108313347


Epoch,Training Loss,Validation Loss


In [16]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1082
  Batch size = 8
100%|██████████| 136/136 [00:40<00:00,  3.39it/s]


{'eval_loss': 0.37423527240753174,
 'eval_accuracy': 0.88909426987061,
 'eval_runtime': 40.4263,
 'eval_samples_per_second': 26.765,
 'eval_steps_per_second': 3.364,
 'epoch': 1.0}

## Adapter

In [17]:
from transformers import BertConfig, AutoAdapterModel,AutoTokenizer

tokenizer_clinical_bio  = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT",model_max_length=150)
model_clinical_adapter = AutoAdapterModel.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT", 
                                                                    num_labels=3,id2label={0: 'PRESENT', 1: 'ABSENT', 2:'POSSIBLE'})


model_clinical_adapter.add_adapter("i2b2-ast")
model_clinical_adapter.train_adapter("i2b2-ast")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/mayerantoine/.cache/huggingface/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b/config.json
Model config BertConfig {
  "_name_or_path": "emilyalsentzer/Bio_Discharge_Summary_BERT",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /Users/mayerantoine/.cache/huggingface/hub/models--emilyal

In [18]:
model_clinical_adapter.add_classification_head("i2b2-ast", num_labels=3)
model_clinical_adapter.set_active_adapters("i2b2-ast")

Adding head 'i2b2-ast' with config {'head_type': 'classification', 'num_labels': 3, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}, 'use_pooler': False, 'bias': True}.


In [19]:
model_clinical_adapter = model_clinical_adapter.to(device)

In [20]:
def tokenize_function(example):
    return tokenizer_clinical_bio(example["sentence"],   padding="max_length", truncation=True)

In [21]:
tokenized_ds = ds.map(tokenize_function, batched=True)
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds = tokenized_ds.remove_columns(["sentence"])
#tokenized_ds = tokenized_ds.remove_columns(["__index_level_0__"])
tokenized_ds.set_format("torch")

Map: 100%|██████████| 602/602 [00:00<00:00, 15993.28 examples/s]


In [22]:
special_tokens_dict = {"additional_special_tokens": ["[entity]"]}
num_added_toks = tokenizer_clinical_bio.add_special_tokens(special_tokens_dict,False)

print("We have added", num_added_toks, "tokens")
# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
model_clinical_adapter.resize_token_embeddings(len(tokenizer_clinical_bio))

Assigning ['[entity]'] to the additional_special_tokens key of the tokenizer


We have added 1 tokens


Embedding(28997, 768)

In [23]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

adaptraining_args = TrainingArguments(
    learning_rate=1e-4,
    evaluation_strategy="epoch",
    num_train_epochs=1,
    output_dir="./clinical_bert_adapter",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
   # remove_unused_columns=False,
)


adaptrainer = AdapterTrainer(
    model=model_clinical_adapter,
    args=adaptraining_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
adaptrainer.train()

/Users/mayerantoine/miniforge3/envs/negation-detection/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4327
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 541
  Number of trainable parameters = 23757123
 92%|█████████▏| 500/541 [07:28<00:37,  1.10it/s]Saving model checkpoint to ./clinical_bert_adapter/checkpoint-500
Configuration saved in ./clinical_bert_adapter/checkpoint-500/i2b2-ast/adapter_config.json
Module weights saved in ./clinical_bert_adapter/checkpoint-500/i2b2-ast/pytorch_adapter.bin
Configuration saved in ./clinical_bert_adapter/checkpoint-

{'loss': 0.4497, 'learning_rate': 7.578558225508318e-06, 'epoch': 0.92}


100%|██████████| 541/541 [08:06<00:00,  1.16it/s]***** Running Evaluation *****
  Num examples = 1082
  Batch size = 8
                                                 
100%|██████████| 541/541 [08:50<00:00,  1.16it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 541/541 [08:50<00:00,  1.02it/s]

{'eval_loss': 0.3326496183872223, 'eval_accuracy': 0.8937153419593346, 'eval_runtime': 44.2612, 'eval_samples_per_second': 24.446, 'eval_steps_per_second': 3.073, 'epoch': 1.0}
{'train_runtime': 530.3413, 'train_samples_per_second': 8.159, 'train_steps_per_second': 1.02, 'train_loss': 0.4394985858261475, 'epoch': 1.0}


TrainOutput(global_step=541, training_loss=0.4394985858261475, metrics={'train_runtime': 530.3413, 'train_samples_per_second': 8.159, 'train_steps_per_second': 1.02, 'train_loss': 0.4394985858261475, 'epoch': 1.0})

In [25]:
model_clinical_adapter.save_all_adapters("./clinical_bert_adapter")

Configuration saved in ./clinical_bert_adapter/i2b2-ast/adapter_config.json
Module weights saved in ./clinical_bert_adapter/i2b2-ast/pytorch_adapter.bin
Configuration saved in ./clinical_bert_adapter/i2b2-ast/head_config.json
Module weights saved in ./clinical_bert_adapter/i2b2-ast/pytorch_model_head.bin
